In [1]:
from MMT_reduce_module import *

In [2]:
from __future__ import print_function
import numpy as np
import scipy as sp
import scipy.ndimage
from astropy.io import fits
import sys
import numpy.ma as ma
import math
import pidly
import os
import glob
import time
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()


## MMT_reduce_module Testing

#### The purpose of this notebook is to test functions in the MMT module one-by-one and examine the outputs at each step.

Major to-do items (April 4, 2019): 

* Fix saturated data!
    * Examine how sky frames are being made 
    * Examine how sky subtraction is being performed on science data
    * Determine best practice of aligning saturated frames
    * plt.imshow reduced steps
    * Stacking size issue 
    
* Determine how flatfield generation is being done and improve if need be
    * Debug negative dark frames/negative flat frame issues? -- **done, 5/23**
    * Double-check dome flat correction, now that twilight flat correction is working (with correct inputs)
    
More to-do items (last updated 5/23):
* Ensure corquad correction is applied prior to analysis steps - **done, 5/23**
* Rotation angle correction and image flip for MMT data (appears to be same VLT conventions) - update within module
* Cross-check sky subtraction in case of very saturated data, and if small changes in rotation angle make difference


### Set up paths to raw data and definitions

In [ ]:
path_to_raw_sci = '/Volumes/Passport4TB/MinMs/HIP12097_MMT_20161012/0.8/'
path_to_raw_darks = '/Volumes/Passport4TB/MinMs/20161012_Calibrations/darks/'
path_to_raw_flats = '/Volumes/Passport4TB/MinMs/20161012_Calibrations/twilight/sky flat/20.0/'
objname = 'HIP12097_Test_0.8s'
flattype = 0 # 0 for sky, 1 for lamp
saturated = 0 # 0 if unsaturated, 1 if saturated
alignflag = 0 # 0 if single star or faint companion, 1 if equal brightness binary
imsize = 1024

## The following cells are each of the steps of the reduce_raw_sci function

In [ ]:
# Make list of science frames and check exposure time 
scilist = glob.glob(path_to_raw_sci + 'q*.fits')

print(f"Number of science frames found: {len(scilist)} \n")


In [ ]:
scitimes = [fits.getheader(im, ignore_missing_end = True)['EXPTIME'] for im in scilist]

# check if all of the exposure times in the current directory are the same:
if all(x == scitimes[0] for x in scitimes):
    print("Science frame exposure time: " + str(scitimes[0]) + "\n")
else:
    raise Exception("Exposure times for given list of files do not match. \
    You may need to make/define separate subfolders for different exptimes.")

sci_exptime = scitimes[0]

n = len(scilist)


# get header from science frames to work with
sciheader = fits.getheader(scilist[0])


In [ ]:
# check for datacubes
if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames
    sciarray = np.zeros([imsize,imsize,n*fits.getdata(scilist[0]).shape[0]])
else:
    sciarray = np.zeros([imsize,imsize,n])

if len(fits.getdata(scilist[0]).shape) == 3: # check for data cubes of science frames    
    totalframes = n*fits.getdata(scilist[0]).shape[0]
else:
    totalframes = n

In [ ]:
# in case data were rotated during observing sequence, set up empty array of rotation angles
angle = np.zeros(totalframes)

im_index = 0

for ii in range(0, n):
    
    im = fits.getdata(scilist[ii], ignore_missing_end=True)
    header = fits.getheader(scilist[ii],ignore_missing_end=True)
    
    if len(im.shape) == 3: # check for data cubes of science frames
        assert not np.any(np.isnan(im))
        for jj in range(0, im.shape[0]):
            sciarray[:,:,im_index] = im[jj,:,:]
            angle[im_index] = (header['PA'] - header['ROT']) * (np.pi/180.0)
            im_index += 1
    else: 
        sciarray[:,:,ii] = im  
        angle[ii] = (header['PA'] - header['ROT']) * (np.pi/180.0)
    header = fits.getheader(scilist[ii], ignore_missing_end=True)





In [ ]:
print(angle)
print(im)

In [ ]:
#MASTER_DARK

print("Creating and applying master darks and flats...\n")    

# create master dark matching science exposure times
med_dark = dark_combine(path_to_raw_darks, sci_exptime, imsize, objname) 

vmin, vmax = interval.get_limits(med_dark)
plt.imshow(med_dark, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:

# subtract off the median dark frame from each of the science frames
for ii in range (0, totalframes):
    sciarray[:,:,ii] -= med_dark


In [ ]:
#MEDIAN_AND_MASTER_FLAT

# create the masterflat 
med_flat, master_flat, flatheader = process_flats(path_to_raw_flats, path_to_raw_darks, imsize, flattype, objname)

vmin, vmax = interval.get_limits(med_flat)
plt.imshow(med_flat, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
vmin, vmax = interval.get_limits(master_flat)
plt.imshow(master_flat, vmin=vmin, vmax=vmax, origin='lower')

In [ ]:

# divide each science frame by the masterflat frame
for ii in range(0, totalframes):
    sciarray[:,:,ii] /= master_flat




In [ ]:
#BAD_PIXEL_MAP

print("Creating bad pixel map and correcting for bad pixels and cosmic rays. \n",
     "This may take a moment... \n") 

# create bad pixel map
badflat = badpixelmap(med_flat, objname, flatheader)  

vmin, vmax = interval.get_limits(badflat)
plt.imshow(badflat, vmin=vmin, vmax=vmax)

In [ ]:

# correct the bad pixels and cosmic rays
reduced_sciarray = correct_bad_pixels(sciarray, badflat)

# print(reduced_sciarray)

In [ ]:
# write out a test reduced science image 
fits.writeto('test_reduced_science.fits', reduced_sciarray[:,:,0], overwrite=True)

In [ ]:
#MASTER_SKY_A_AND_B
print("Creating master sky from science frames...\n") 

# create median sky from stack of science images
sky_output = create_sky_frames(reduced_sciarray, sciheader, objname, angle)



In [ ]:
# get median and examine sky output:
print(np.median(sky_output[1]))
vmin, vmax = interval.get_limits(sky_output[1])
plt.imshow(sky_output[1], vmin=vmin, vmax=vmax, origin='lower')

In [ ]:
sky_output

In [ ]:
# apply sky subtraction to each science image 
skysub_science_array, rot_flag = sky_subtract(reduced_sciarray, sky_output, angle)

In [ ]:
rot_flag

In [ ]:
#INDIVIDUAL_REDUCED_SCI_IMAGES
t0=time.time()

# initialize blank list to hold all of the reduced science image names
scinames_list = []


for ii in range(0, totalframes):
    print(f"Saving reduced frame #{ii}")
    sciname = 'reducedsci_00' + str(ii) + '.fits'
    if ii >= 10:
        sciname = 'reducedsci_0' + str(ii) + '.fits'
    if ii >= 100:
        sciname = 'reducedsci_' + str(ii) + '.fits'
    fits.writeto(sciname, skysub_science_array[:,:,ii], sciheader, overwrite = True, output_verify='silentfix')
    scinames_list.append(sciname)
    
t1=time.time()
print("Time taken: ", (t1-t0)/60.)

In [3]:
!export XPA_METHOD='unix'

In [3]:
idl = pidly.IDL('/Applications/exelis/idl85/bin/idl')

In [4]:
idl('ds9')

% Compiled module: DS9.
Spawning ds9 with title: ds9_1558666634
waiting...
waiting...
waiting...
it worked!


In [5]:
# get current directory where reduced frames are written
current_dir = os.getcwd()

# measure star positions in all of the images
xcen, ycen = measure_star_centers(skysub_science_array, scinames_list, sciheader, saturated, alignflag, current_dir, saveframes = True)


NameError: name 'skysub_science_array' is not defined

# everything below is previous testing

In [ ]:
!export XPA_METHOD='unix'

In [ ]:
# get current directory where reduced frames are written
current_dir = os.getcwd()

# measure star positions in all of the images
xcen, ycen = measure_star_centers(skysub_science_array[:,:,1:3], scinames_list[1:3], sciheader, saturated, alignflag, current_dir, saveframes = True)

In [ ]:
print('xcen:', xcen, 'ycen:', ycen)

In [ ]:
#SHIFTED_IMAGES
#STACKED
#FINAL
t0=time.time()
# final step (!) - shift and combine all of the images.
rotate_shift_align(xcen, ycen, angle, skysub_science_array[:,:,1:3], objname, sciheader, current_dir, imsize=1024)

t1 = time.time()
timetaken = (t1-t0)/60.
print(f"Completed reduction of {totalframes} images in {timetaken} minutes.")






In [ ]:
reduced_array = glob.glob('reduced*')

for ii in reduced_array:
    reducedsci = fits.getdata(ii)
    reducedsci_header = fits.getheader(ii)
    
    xcen = reducedsci_header['CRPIX1A']
    ycen = reducedsci_header['CRPIX2A']
    
    vmin1, vmax1 = interval.get_limits(reducedsci)
    plt.imshow(reducedsci, vmin=vmin1, vmax=vmax1)
    plt.plot(xcen, ycen, marker = 'o', markersize = 10)
    plt.show()



In [ ]:
n = len(scinames_list)

xcen = np.zeros(n)
ycen = np.zeros(n)

idl = pidly.IDL('/Applications/exelis/idl/bin/idl')
idl_changedir = 'cd, ' + f'"{current_dir}"'
idl(idl_changedir)

for ii in range(0, 1):
    # idl('name = "'+sciname+'"')
    idl('name = "'+ scinames_list[ii] +'"')
    idl('im=MRDFITS(name,0,/FSCALE,/SILENT)')
    idl('tmp=SMOOTH(im,21,/EDGE_TRUNCATE)')
    idl('tmp[0:100,*]=0.0')
    idl('tmp[924:1023,*]=0.0')
    idl('tmp[*,0:100]=0.0')
    idl('tmp[*,1000:1023]=0.0')
    idl('foo=MAX(tmp,ind,/NAN)')
    idl('ind=ARRAY_INDICES(tmp,ind)')
    idl('print, foo')
    
    if saturated == 0:
        idl('GCNTRD,im[ind[0]-20:ind[0]+20,ind[1]-20:ind[1]+20],20,20,xcen,ycen,3.0')
        idl('xcen += ind[0]-20.0')
        idl('ycen += ind[1]-20.0')
        xcen[ii], ycen[ii] = idl.xcen, idl.ycen
    else:
        idl('x=ind[0]')
        idl('y=ind[1]')
        idl('sim = im[x-20:x+20,y-20:y+20]')
        idl('weights = (sim*0.0)+1.0')
        idl('weights[WHERE(sim ge 0.7*MAX(sim,/NAN))]=0.0')
        idl('fit=MPFIT2DPEAK(sim,A,WEIGHTS=weights)')
        idl('xcen = A[4]+(x-20.0)')
        idl('ycen = A[5]+(y-20.0)')
        xcen[ii], ycen[ii] = idl.xcen, idl.ycen
        idl('print, A[5]')

print(xcen[ii],ycen[ii])

In [ ]:
idl.A

In [ ]:

reduce_raw_sci(path_to_raw_sci, path_to_raw_darks, path_to_raw_flats, objname, flattype, saturated, alignflag, imsize = 1024)